#  Tone Classification Task

## Data Preparation

In [204]:
require 'loader'
require 'nn'
MAX_LEN = 256
max_len = 64
-- Loader:new(obj, max_len)
ldr = Loader:new(nil, MAX_LEN)

In [205]:
function moving_avg(vec, window)
    local len = (#vec)[1]
    local new_vec = torch.Tensor(len):fill(0.0)
    for i = 1, (len - window - 1) do
        local sum = vec[{{i, i + window - 1}}]:sum() / window
        new_vec[i+1] = sum
    end
    return new_vec
end

In [206]:
-- kick too large or too small values
function kick(vec, topsmall, toplarge)
    -- kick the smalls
    for i = 1, topsmall do
        vec:csub(vec:min())
        new_vec = vec[vec:gt(0.0)]:clone()
        vec = new_vec
    end
    for i = 1, toplarge do
        _, inds = torch.max(vec, 1)
--         print(inds)
        for j = 1, (#inds)[1] do
            vec[inds[j]] = 0.0
        end
        new_vec = vec[vec:gt(0.0)]:clone()
        vec = new_vec
    end
    return new_vec
end

-- local i = 0
-- local tester = torch.Tensor(10):apply(function() i = i + 1 return i end)
-- print(tester)
-- print(kick(tester, 1, 0))

In [207]:
local matrix = require "matrix"
local fit = {}
local function getresults( mtx )
    assert( #mtx+1 == #mtx[1], "Cannot calculate Results" )
    mtx:dogauss()
    -- tresults
    local cols = #mtx[1]
    local tres = {}
    for i = 1,#mtx do
        tres[i] = mtx[i][cols]
    end
    return unpack( tres )
end

function fit.parabola( x_values,y_values )
    -- x_values = { x1,x2,x3,...,xn }
    -- y_values = { y1,y2,y3,...,yn }

    -- values for A matrix
    local a_vals = {}
    -- values for Y vector
    local y_vals = {}

    for i,v in ipairs( x_values ) do
        a_vals[i] = { 1, v, v*v }
        y_vals[i] = { y_values[i] }
    end

    -- create both Matrixes
    local A = matrix:new( a_vals )
    local Y = matrix:new( y_vals )

    local ATA = matrix.mul( matrix.transpose(A), A )
    local ATY = matrix.mul( matrix.transpose(A), Y )

    local ATAATY = matrix.concath(ATA,ATY)

    return getresults( ATAATY )
end

function fit_quad(vec, targetLen)
    local len = (#vec)[1]
    local xs = {}
    local ys = {}
    local a, b, c
    for i = 1, len do
        table.insert(xs, i)
        table.insert(ys, vec[i])
    end
    a, b, c = fit.parabola(xs, ys)
    local res = torch.Tensor(targetLen)
    for i = 1, targetLen do
        local ii = (i - 1) / (targetLen - 1) * (len - 1) + 1
        res[i] = a + b * ii + c * ii * ii
    end
    return res
end

In [217]:
function make_linear_spine(vec, targetLen)
    local len = (#vec)[1]
    local ys = {}
    
    for i = 1, len do
        table.insert(ys, vec[i])
    end
    
    local res = torch.Tensor(targetLen)
    
    local i = 1
    for ind = 1, targetLen do
        local x = (ind - 1) * (len - 1) / (targetLen - 1) + 1
        if  x > i then
            i = i + 1
        end
        if x == i then 
            res[ind] = ys[i]
        else
            res[ind] = (ys[i] - ys[i - 1]) * (x - i + 1) + ys[i - 1]
        end
    end
    return res
end

# Dataset!

In [230]:
function make_dataset(name)
    local dataset = {data = ldr.data[name], label = ldr.label[name]}
    function dataset:size() 
        return self.data:size(1) 
    end
    for i = 1, 2 do
        for j = 1, dataset:size() do
            local std = dataset.data[{j, {}, i}]:std()
            dataset.data[{j, {}, i}]:div(std)
        end
    end
    for i = 1, 2 do
        local std = dataset.data[{{}, {}, i}]:std()
        dataset.data[{{}, {}, i}]:div(std)
    end
    local resized_collection = {
        data = torch.Tensor(dataset:size(), max_len,2),
        label = dataset.label
    }
    function resized_collection:size()
        return self.data:size(1) 
    end
    for i = 1, dataset:size() do
        for j = 1, MAX_LEN do
--             if (dataset.data[{i, j, 2}] < 2.0 or dataset.data[{i, j, 1}] < 1.35) then
            if (dataset.data[{i, j, 2}] < 2.0) then
                dataset.data[{i, j, 1}] = 0.0
                dataset.data[{i, j, 2}] = 0.0
            end
        end
        
        
        local shifted_F0 = torch.Tensor(max_len):fill(0.0)
        local shifted_Engy = torch.Tensor(max_len):fill(0.0)
        local data = dataset.data[{i, {}, {}}]
        local tmpF0 = data[{{}, 1}][data[{{}, 1}]:gt(0.0)]:clone()
        local tmpEngy = data[{{}, 2}][data[{{}, 2}]:gt(0.0)]:clone()
        
        
--         tmpF0 = kick(tmpF0, 1, 1)
--         tmpEngy = kick(tmpEngy, 0, 0)
        
        tmpF0 = moving_avg(tmpF0, 3)
        tmpF0 = kick(tmpF0, 1, 0)
        
        tmpEngy = moving_avg(tmpEngy, 5)
        tmpEngy = kick(tmpEngy, 1, 0)
        
--         shifted_F0[{{1, (#tmpF0)[1]}}] = tmpF0, 5
--         shifted_Engy[{{1, (#tmpEngy)[1]}}] = tmpEngy
        
--         resized_collection.data[{i, {}, 1}] = shifted_F0
--         resized_collection.data[{i, {}, 2}] = shifted_Engy
        
        resized_collection.data[{i, {}, 1}] = make_linear_spine(tmpF0, max_len)
        peek = tmpF0
        resized_collection.data[{i, {}, 2}] = make_linear_spine(tmpEngy, max_len)
    
    
    end
--     for i = 1, 2 do
--         for j = 1, dataset:size() do
--             local std = resized_collection.data[{j, {}, i}]:std()
--             resized_collection.data[{j, {}, i}]:div(std)
--         end
--     end
--     for i = 1, 2 do
--         local std = resized_collection.data[{{}, {}, i}]:std()
--         resized_collection.data[{{}, {}, i}]:div(std)
--     end
    
    resized_collection.data[{{}, {}, 2}]:fill(0.0)
    for i = 1, dataset:size() do
        resized_collection[i] = {resized_collection.data[i], resized_collection.label[i]}
    end
    print("dataset "..name.." has been constructed!")
    return resized_collection
end

In [231]:
trainset = make_dataset('train')
testset = make_dataset('test')
newtestset = make_dataset('test_new')

dataset train has been constructed!	


dataset test has been constructed!	


dataset test_new has been constructed!	


In [232]:
Plot = require 'itorch.Plot'
local x1 = torch.range(1,max_len)
local sampleN = 57
local y1 = newtestset.data[{sampleN,{},1}]
local y2 = newtestset.data[{sampleN,{},2}]
-- local x1 = torch.range(1,18)
-- local y1 = peek
plot = Plot():line(x1, y1,'red','f0'):legend(true):title('Feature Observation of a Tone '..newtestset.label[sampleN]):draw()
plot:line(x1, y2,'blue','engy'):redraw()

## Model

In [233]:
require 'dp'
local inp = 2;  -- dimensionality of one sequence element 
local outp = 4; -- number of derived features for one sequence element
local conv_kw = 4;   -- number of sequence element kernel operates on per step
local conv_dw = 4;   -- step dw and go on to the next sequence element

local pool_kw = 4;   -- number of sequence element pooled on per step
local pool_dw = 4;   -- step dw and go on to the next sequence element

local nOutputFrame = (max_len - conv_kw) / conv_dw + 1
local nOutputFrame = (nOutputFrame - pool_kw) / pool_dw + 1

net = nn.Sequential()
peek = nn.TemporalConvolution(inp, outp, conv_kw, conv_dw)
net:add(peek)
net:add(nn.ReLU())
net:add(nn.TemporalMaxPooling(pool_kw, pool_dw))
net:add(nn.View(outp*nOutputFrame))
net:add(nn.Linear(outp*nOutputFrame, 128))
-- net:add(nn.BatchNormalization(64))
net:add(nn.ReLU())
net:add(nn.Linear(128, 4))
net:add(nn.LogSoftMax())

In [234]:
print (net:__tostring())

nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> output]
  (1): nn.TemporalConvolution
  (2): nn.ReLU
  (3): nn.TemporalMaxPooling
  (4): nn.View(16)
  (5): nn.Linear(16 -> 128)
  (6): nn.ReLU
  (7): nn.Linear(128 -> 4)
  (8): nn.LogSoftMax
}	


In [239]:
criterion = nn.ClassNLLCriterion();  
trainer = nn.StochasticGradient(net, criterion)  
trainer.learningRate = 0.001
trainer.maxIteration = 200
trainer.verbose = False

In [240]:
function cal_acc(dataset)
    local acc = 0.0
    for i = 1, dataset:size() do
        _, output = torch.max(net:forward(dataset[i][1]), 1)
        if output[1] == dataset[i][2] then
            acc = acc + 1.0
        end
    end
    acc = acc / dataset:size()
    return acc
end

In [241]:
recorder = {loss = {}, trainacc = {}, valacc = {}, testacc = {}}
trainer.hookIteration = function(obj, iteration, currentError) 
    local trainacc = cal_acc(trainset)
    local valacc = cal_acc(testset)
    local testacc = cal_acc(newtestset)
    table.insert(recorder.loss, currentError)
    table.insert(recorder.trainacc, trainacc)
    table.insert(recorder.valacc, valacc)
    table.insert(recorder.testacc, testacc)
    if (iteration % 10 == 0) then 
        print('Iteration '..iteration..': loss('..currentError..') train_acc('
            ..trainacc..') test_acc('..valacc..') new_test_acc('..testacc..').')
    end
end

In [242]:
trainer:train(trainset)

# StochasticGradient: training	


Iteration 10: loss(0.28919479782248) train_acc(0.9375) test_acc(0.95) new_test_acc(0.73245614035088).	


Iteration 20: loss(0.28021395291023) train_acc(0.9375) test_acc(0.95) new_test_acc(0.73684210526316).	


Iteration 30: loss(0.27176185013533) train_acc(0.94) test_acc(0.95) new_test_acc(0.75).	


Iteration 40: loss(0.26450652715343) train_acc(0.945) test_acc(0.95) new_test_acc(0.75).	


...

Iteration 160: loss(0.21794129024669) train_acc(0.95) test_acc(0.95) new_test_acc(0.76315789473684).	


Iteration 170: loss(0.21617859049863) train_acc(0.95) test_acc(0.95) new_test_acc(0.76315789473684).	


Iteration 180: loss(0.21458903595126) train_acc(0.9525) test_acc(0.95) new_test_acc(0.75877192982456).	


Iteration 190: loss(0.21258571923968) train_acc(0.955) test_acc(0.95) new_test_acc(0.76315789473684).	


Iteration 200: loss(0.21108299943349) train_acc(0.9525) test_acc(0.95) new_test_acc(0.75877192982456).	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 0.21108299943349	



In [ ]:
local x = torch.range(1, #recorder.loss)
plt = Plot():line(x, recorder.loss,'red','loss'):legend(true):title('Train Loss'):draw()
plt = Plot():line(x, recorder.trainacc,'blue','train_acc'):legend(true):title('Accuracy'):draw()
plt:line(x, recorder.valacc,'green','val_acc'):redraw()

In [21]:
print ('accuracy on train set '..cal_acc(trainset))
print ('accuracy on test set '..cal_acc(testset))
print ('accuracy on new test set '..cal_acc(newtestset))

accuracy on train set 0.9525	


accuracy on test set 0.95	


accuracy on new test set 0.78947368421053	


## To see why

In [22]:
mislabeled = {}
local output
for i = 1, newtestset:size() do
    _, output = torch.max(net:forward(newtestset[i][1]), 1)
    if output[1] ~= newtestset[i][2] then
        table.insert(mislabeled, { data_id = i, pred = output[1], gt = newtestset[i][2]})
    end
end

In [23]:
print(#mislabeled)

48	


In [24]:
print(mislabeled)

{
  1 : 
    {
      data_id : 49
      pred : 4
      gt : 1
    }
  2 : 
    {
      data_id : 53
      pred : 4
      gt : 1
    }
  3 : 
    {
      data_id : 57
      pred : 3
      gt : 2
    }
  4 : 
    {
      data_id : 58
      pred : 3
      gt : 2
    }
  5 : 
    {
      data_id : 59
      pred : 3
      gt : 2
    }
  6 : 
    {
      data_id : 60
      pred : 3
      gt : 2
    }
  7 : 
    {
      data_id : 68
      pred : 1
      gt : 2
    }
  8 : 
    {
      data_id : 70
      pred : 1
      gt : 2
    }
  9 : 
    {
      data_id : 71
      pred : 1


      gt : 2
    }
  10 : 
    {
      data_id : 72
      pred : 1
      gt : 2
    }
  11 : 
    {
      data_id : 73
      pred : 4


      gt : 2
    }
  12 : 
    {
      data_id : 76
      pred : 4
      gt : 2
    }
  13 : 
    {
      data_id : 92
      pred : 1
      gt : 2
    }
  14 : 
    {
      data_id : 93
      pred : 1
      gt : 2
    }
  15 : 
    {
      data_id : 96
      pred : 1
      gt : 2
    }
  16 : 
    {
      data_id : 98
      pred : 1
      gt : 2
    }
  17 : 
    {
      data_id : 101
      pred : 1
      gt : 2
    }
  18 : 
    {
      data_id : 104
      pred : 1
      gt : 2
    }
  19 : 
    {
      data_id : 105
      pred : 1
      gt : 2
    }
  20 : 
    {
      data_id : 108
      pred : 3
      gt : 2
    }
  21 : 
    {
      data_id : 109
      pred : 1
      gt : 2
    }
  22 : 
    {
      data_id : 113
      pred : 4
      gt : 2
    }
  23 : 
    {
      data_id : 119
      pred : 4
      gt : 3
    }
  24 : 
    {
      data_id : 123
      pred : 2
      gt : 3
    }
  25 : 
    {
      data_id : 124
      pred : 2
      gt : 3
    }
  26 : 
    {
      data_id : 125


    }
  28 : 
    {
      data_id : 132
      pred : 2
      gt : 3
    }
  29 : 
    {
      data_id : 137
      pred : 1
      gt : 3
    }
  30 : 
    {
      data_id : 140
      pred : 1
      gt : 3
    }
  31 : 
    {
      data_id : 144
      pred : 1
      gt : 3
    }
  32 : 
    {
      data_id : 152
      pred : 1
      gt : 3
    }
  33 : 
    {
      data_id : 159
      pred : 4
      gt : 3
    }
  34 : 
    {
      data_id : 160
      pred : 2
      gt : 3
    }
  35 : 
    {
      data_id : 161
      pred : 1
      gt : 3
    }
  36 : 
    {
      data_id : 162
      pred : 2
      gt : 3
    }
  37 : 
    {
      data_id : 171
      pred : 2
      gt : 3
    }
  38 : 
    {
      data_id : 172
      pred : 2
      gt : 3
    }
  39 : 
    {
      data_id : 174
      pred : 2
      gt : 3
    }
  40 : 
    {
      data_id : 175
      pred : 3
      gt : 4
    }
  41 : 
    {
      data_id : 185
      pred : 3
      gt : 4
    }
  42 : 
    {
      data_id : 193


      pred : 1
      gt : 4
    }
  43 : 
    {
      data_id : 194
      pred : 2
      gt : 4
    }
  44 : 
    {
      data_id : 195
      pred : 3
      gt : 4
    }
  45 : 
    {
      data_id : 198
      pred : 2
      gt : 4
    }
  46 : 
    {
      data_id : 199
      pred : 1
      gt : 4
    }
  47 : 
    {
      data_id : 218
      pred : 1
      gt : 4
    }
  48 : 
    {
      data_id : 222
      pred : 1
      gt : 4
    }
}


In [ ]:
print(net:forward(newtestset[4][1]))
-- print(peek:forward(newtestset[4][1]))
print(peek.weight)
-- local x1 = torch.range(1, MAX_LEN - 1)
-- local sampleN = 7
-- local y1 = newtestset.data[{sampleN,{},1}]
-- local y2 = peek
-- plot = Plot():line(x1, y1,'red','f0'):legend(true):title('Feature Observation of a Tone '..newtestset.label[sampleN]):draw()
-- plot:line(x1, y2,'blue','engy'):redraw()

## Try Rules!

In [ ]:
function cal_acc_rule(dataset)
    local acc = 0.0
    for i = 1, dataset:size() do
        output = simple_rule(dataset[i][1])
        if output == dataset[i][2] then
            acc = acc + 1.0
        end
    end
    acc = acc / dataset:size()
    return acc
end

In [ ]:
function simple_rule(example)
    -- only take f0
    local f0 = example[{{}, 1}]
    local max = f0:max()
    if max < 0.3 then return 1 end
    local totcnt = 0
    local cnt = 0
    for i = 1, (#f0)[1] - 1 do
        if f0[i + 1] ~= 0 then totcnt = totcnt + 1 end
        if f0[i + 1] - f0[i] < 0 then cnt = cnt + 1 end
    end
    if cnt > 0.8 * totcnt then return 4
    elseif cnt < 0.2 * totcnt then return 2
    else return 3 end
end

In [ ]:
print ('accuracy on train set '..cal_acc_rule(trainset))
print ('accuracy on test set '..cal_acc_rule(testset))
print ('accuracy on new test set '..cal_acc_rule(newtestset))

In [ ]:
mislabeled_rule = {}
local output
for i = 1, newtestset:size() do
    local output = simple_rule(newtestset[i][1])
    if output ~= newtestset[i][2] then
        table.insert(mislabeled_rule, { data_id = i, pred = output, gt = newtestset[i][2]})
    end
end

In [ ]:
print(#mislabeled_rule)

In [ ]:
print(mislabeled_rule)

In [ ]:
local i = 1.5
local x = torch.Tensor{i, i * i}:reshape(1,2)
print(x)

In [215]:
function make_linear_spine(vec, targetLen)
    local len = (#vec)[1]
    local ys = {}
    
    for i = 1, len do
        table.insert(ys, vec[i])
    end
    
    local res = torch.Tensor(targetLen)
    
    local i = 1
    for ind = 1, targetLen do
        local x = (ind - 1) * (len - 1) / (targetLen - 1) + 1
        if  x > i then
            i = i + 1
        end
        if x == i then 
            res[ind] = ys[i]
        else
            res[ind] = (ys[i] - ys[i - 1]) * (x - i + 1) + ys[i - 1]
        end
    end
    return res
end

In [216]:
-- print(peek)
print(make_linear_spine(peek, 64))
-- fit_quad(peek)

 2.2656
 2.1000
 1.9344
 1.7688
 1.8318
 2.0003
 2.1688
 2.3338
 2.4926
 2.6513
 2.8094
 2.9550
 3.1005
 3.2460
 3.2617
 3.2310
 3.2004
 3.1677
 3.1324
 3.0971
 3.0625
 3.0348
 3.0071
 2.9794
 2.9581
 2.9385
 2.9190
 2.9036
 2.8928
 2.8820
 2.8709
 2.8581
 2.8452
 2.8323
 2.8147
 2.7963
 2.7778
 2.7568
 2.7334
 2.7101
 2.6870
 2.6644
 2.6419
 2.6193
 2.5979
 2.5766
 2.5553
 2.5295
 2.5003
 2.4711
 2.4425
 2.4154
 2.3884
 2.3613
 2.3301
 2.2987
 2.2672
 2.2400
 2.2153
 2.1905
 2.1653
 2.1390
 2.1127
 2.0864
[torch.DoubleTensor of size 64]

